In [1]:
import pandas as pd
import numpy as np


detailed_reg = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv')
cities = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/Cities.csv')
conf_abbr = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/Conferences.csv')
conf_tourney = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MConferenceTourneyGames.csv')
game_cities = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MGameCities.csv')
massey = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MMasseyOrdinals_thruSeason2024_day128.csv')
detailed_tourney = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyDetailedResults.csv')
team_confs = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MTeamConferences.csv')


In [2]:
detailed_reg['Type'] = 'Season'
detailed_tourney['Type'] = 'Tourney'

combined_data = pd.concat([detailed_reg, detailed_tourney], ignore_index=True)

In [3]:
combined_data

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Type
0,2003,10,1104,68,1328,62,N,0,27,58,...,16,22,10,22,8,18,9,2,20,Season
1,2003,10,1272,70,1393,63,N,0,26,62,...,9,20,20,25,7,12,8,6,16,Season
2,2003,11,1266,73,1437,61,N,0,24,58,...,14,23,31,22,9,12,2,5,23,Season
3,2003,11,1296,56,1457,50,N,0,18,38,...,8,15,17,20,9,19,4,3,23,Season
4,2003,11,1400,77,1208,71,N,0,30,61,...,17,27,21,15,12,10,7,1,14,Season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114551,2023,146,1274,88,1400,81,N,0,29,49,...,11,15,9,14,20,13,6,1,23,Tourney
114552,2023,146,1361,57,1166,56,N,0,25,66,...,10,11,8,24,9,9,3,2,11,Tourney
114553,2023,152,1163,72,1274,59,N,0,28,57,...,12,12,12,17,10,9,8,1,12,Tourney
114554,2023,152,1361,72,1194,71,N,0,25,57,...,16,21,7,24,6,9,6,2,17,Tourney


In [4]:

win_columns = {
    'WTeamID': 'TeamID', 'WScore': 'Score', 'LTeamID': 'OpponentID', 'LScore': 'OpponentScore',
    'WFGM': 'FGM', 'WFGA': 'FGA', 'WFGM3': 'FGM3', 'WFGA3': 'FGA3', 'WFTM': 'FTM', 'WFTA': 'FTA',
    'WOR': 'OR', 'WDR': 'DR', 'WAst': 'Ast', 'WTO': 'TO', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF',
    'LFGM': 'OpponentFGM', 'LFGA': 'OpponentFGA', 'LFGM3': 'OpponentFGM3', 'LFGA3': 'OpponentFGA3',
    'LFTM': 'OpponentFTM', 'LFTA': 'OpponentFTA', 'LOR': 'OpponentOR', 'LDR': 'OpponentDR', 
    'LAst': 'OpponentAst', 'LTO': 'OpponentTO', 'LStl': 'OpponentStl', 'LBlk': 'OpponentBlk', 'LPF': 'OpponentPF',
    'WLoc': 'Loc', 'NumOT': 'OT'
}

loss_columns = {
    'LTeamID': 'TeamID', 'LScore': 'Score', 'WTeamID': 'OpponentID', 'WScore': 'OpponentScore',
    'LFGM': 'FGM', 'LFGA': 'FGA', 'LFGM3': 'FGM3', 'LFGA3': 'FGA3', 'LFTM': 'FTM', 'LFTA': 'FTA',
    'LOR': 'OR', 'LDR': 'DR', 'LAst': 'Ast', 'LTO': 'TO', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF',
    'WFGM': 'OpponentFGM', 'WFGA': 'OpponentFGA', 'WFGM3': 'OpponentFGM3', 'WFGA3': 'OpponentFGA3',
    'WFTM': 'OpponentFTM', 'WFTA': 'OpponentFTA', 'WOR': 'OpponentOR', 'WDR': 'OpponentDR', 
    'WAst': 'OpponentAst', 'WTO': 'OpponentTO', 'WStl': 'OpponentStl', 'WBlk': 'OpponentBlk', 'WPF': 'OpponentPF',
    'WLoc': 'Loc', 'NumOT': 'OT'
}

# Apply the renaming for wins and losses
wins_normalized = combined_data.rename(columns=win_columns)
losses_normalized = combined_data.rename(columns=loss_columns)

# Adjust the location for losses_normalized to reflect the perspective of the losing team
losses_normalized['Loc'] = losses_normalized['Loc'].replace({'H': 'A', 'A': 'H'})

# Concatenate the normalized dataframes
all_games_normalized = pd.concat([wins_normalized, losses_normalized], ignore_index=True)

# Sort by TeamID, Season, and DayNum to ensure chronological order for cumulative calculations
all_games_normalized.sort_values(by=['TeamID', 'Season', 'DayNum'], inplace=True)

# From here, you can proceed with calculating cumulative statistics, ensuring to group by 'TeamID' and 'Season'
# and then calculating the stats you're interested in (e.g., cumulative averages for scores, field goals, etc.)
all_games_normalized["FGPct"] = all_games_normalized["FGM"] / all_games_normalized["FGA"]
all_games_normalized["FG3Pct"] = all_games_normalized["FGM3"] / all_games_normalized["FGA3"]
all_games_normalized["FTPct"] = all_games_normalized["FTM"] / all_games_normalized["FTA"]
all_games_normalized["OpponentFGPct"] = all_games_normalized["OpponentFGM"] / all_games_normalized["OpponentFGA"]
all_games_normalized["OpponentFG3Pct"] = all_games_normalized["OpponentFGM3"] / all_games_normalized["OpponentFGA3"]
all_games_normalized["OpponentFTPct"] = all_games_normalized["OpponentFTM"] / all_games_normalized["OpponentFTA"]
import numpy as np

# Estimate possessions for both team and opponent
all_games_normalized['Possessions'] = all_games_normalized['FGA'] - all_games_normalized['OR'] + all_games_normalized['TO'] + (0.475 * all_games_normalized['FTA'])
all_games_normalized['OpponentPossessions'] = all_games_normalized['OpponentFGA'] - all_games_normalized['OpponentOR'] + all_games_normalized['OpponentTO'] + (0.475 * all_games_normalized['OpponentFTA'])

# Offensive and Defensive Efficiency
all_games_normalized['OE'] = (all_games_normalized['Score'] / all_games_normalized['Possessions']) * 100
all_games_normalized['DE'] = (all_games_normalized['OpponentScore'] / all_games_normalized['OpponentPossessions']) * 100

# True Shooting Percentage
all_games_normalized['TSP'] = all_games_normalized['Score'] / (2 * (all_games_normalized['FGA'] + 0.475 * all_games_normalized['FTA']))

# Effective Field Goal Percentage
all_games_normalized['eFG%'] = (all_games_normalized['FGM'] + 0.5 * all_games_normalized['FGM3']) / all_games_normalized['FGA']

# Turnover Rate
all_games_normalized['TOR'] = (all_games_normalized['TO'] / all_games_normalized['Possessions']) * 100

# Offensive and Defensive Rebound Rates
all_games_normalized['ORR'] = all_games_normalized['OR'] / (all_games_normalized['OR'] + all_games_normalized['OpponentDR'])
all_games_normalized['DRR'] = all_games_normalized['DR'] / (all_games_normalized['OpponentOR'] + all_games_normalized['DR'])

# Assist Ratio
all_games_normalized['AR'] = (all_games_normalized['Ast'] / all_games_normalized['Possessions']) * 100

# Block Rate
all_games_normalized['BlockRate'] = all_games_normalized['Blk'] / (all_games_normalized['OpponentFGA'] - all_games_normalized['OpponentFGA3'])

# Steal Rate
all_games_normalized['StealRate'] = (all_games_normalized['Stl'] / all_games_normalized['OpponentPossessions']) * 100

# Free Throw Rate
all_games_normalized['FTR'] = all_games_normalized['FTA'] / all_games_normalized['FGA']

# Net Rating
all_games_normalized['NetRating'] = all_games_normalized['OE'] - all_games_normalized['DE']

# Calculate additional basketball metrics

# Usage Rate
all_games_normalized['USG%'] = ((all_games_normalized['FGA'] + 0.475 * all_games_normalized['FTA'] + all_games_normalized['TO']) * 100) / all_games_normalized['Possessions']

# Points Per Shot
all_games_normalized['PPS'] = all_games_normalized['Score'] / (all_games_normalized['FGA'] + 0.475 * all_games_normalized['FTA'])

# Defensive Rebound Percentage
all_games_normalized['DRB%'] = all_games_normalized['DR'] / (all_games_normalized['DR'] + all_games_normalized['OpponentOR'])

# Offensive Rebound Percentage
all_games_normalized['ORB%'] = all_games_normalized['OR'] / (all_games_normalized['OR'] + all_games_normalized['OpponentDR'])




In [5]:
all_games_normalized.columns

Index(['Season', 'DayNum', 'TeamID', 'Score', 'OpponentID', 'OpponentScore',
       'Loc', 'OT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OpponentFGM', 'OpponentFGA',
       'OpponentFGM3', 'OpponentFGA3', 'OpponentFTM', 'OpponentFTA',
       'OpponentOR', 'OpponentDR', 'OpponentAst', 'OpponentTO', 'OpponentStl',
       'OpponentBlk', 'OpponentPF', 'Type', 'FGPct', 'FG3Pct', 'FTPct',
       'OpponentFGPct', 'OpponentFG3Pct', 'OpponentFTPct', 'Possessions',
       'OpponentPossessions', 'OE', 'DE', 'TSP', 'eFG%', 'TOR', 'ORR', 'DRR',
       'AR', 'BlockRate', 'StealRate', 'FTR', 'NetRating', 'USG%', 'PPS',
       'DRB%', 'ORB%'],
      dtype='object')

In [6]:
stats_of_interest = ['Score', 'OpponentID', 'OpponentScore',
       'OT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OpponentFGM', 'OpponentFGA',
       'OpponentFGM3', 'OpponentFGA3', 'OpponentFTM', 'OpponentFTA',
       'OpponentOR', 'OpponentDR', 'OpponentAst', 'OpponentTO', 'OpponentStl',
       'OpponentBlk', 'OpponentPF', 'FGPct', 'FG3Pct', 'FTPct',
       'OpponentFGPct', 'OpponentFG3Pct', 'OpponentFTPct', 'Possessions',
       'OpponentPossessions', 'OE', 'DE', 'TSP', 'eFG%', 'TOR', 'ORR',
       'DRR', 'AR', 'BlockRate', 'StealRate', 'FTR', 'NetRating', 'USG%', 'PPS', 'DRB%', 'ORB%']
for stat in stats_of_interest:
    all_games_normalized[f'CumulativeSum_{stat}'] = all_games_normalized.groupby(['TeamID', 'Season'])[stat].cumsum()

# Calculate the game count to use for average calculations
all_games_normalized['GameCount'] = all_games_normalized.groupby(['TeamID', 'Season']).cumcount() + 1

# Calculate cumulative averages for the statistics
for stat in stats_of_interest:
    all_games_normalized[f'CumulativeAvg_{stat}'] = all_games_normalized[f'CumulativeSum_{stat}'] / all_games_normalized['GameCount']


/tmp/ipykernel_19/2845489207.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_games_normalized[f'CumulativeSum_{stat}'] = all_games_normalized.groupby(['TeamID', 'Season'])[stat].cumsum()
/tmp/ipykernel_19/2845489207.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_games_normalized[f'CumulativeSum_{stat}'] = all_games_normalized.groupby(['TeamID', 'Season'])[stat].cumsum()
/tmp/ipykernel_19/2845489207.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

In [7]:
all_games_normalized

,Season,DayNum,TeamID,Score,OpponentID,OpponentScore,Loc,OT,FGM,FGA,...,CumulativeAvg_DRR,CumulativeAvg_AR,CumulativeAvg_BlockRate,CumulativeAvg_StealRate,CumulativeAvg_FTR,CumulativeAvg_NetRating,CumulativeAvg_USG%,CumulativeAvg_PPS,CumulativeAvg_DRB%,CumulativeAvg_ORB%
169822,2014,5,1101,75,1182,94,A,0,22,63,...,0.687500,11.658031,0.025641,3.969567,0.507937,-27.229496,114.248705,0.959079,0.687500,0.255814
169892,2014,7,1101,47,1382,75,A,0,14,40,...,0.586607,11.726235,0.024725,3.592500,0.578968,-34.308518,116.391412,0.928441,0.586607,0.294574
169982,2014,9,1101,44,1268,67,A,0,17,54,...,0.573867,12.504990,0.016484,5.516342,0.385979,-34.810666,114.052608,0.890566,0.573867,0.256988
170149,2014,13,1101,41,1234,103,A,0,14,54,...,0.630400,11.562149,0.012363,5.186942,0.345040,-47.227246,114.178466,0.839616,0.630400,0.253717
170249,2014,15,1101,64,1395,71,A,0,19,50,...,0.657261,13.320169,0.021319,5.407910,0.376032,-40.078482,114.787000,0.878562,0.657261,0.264085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226820,2024,110,1478,58,1192,68,H,0,21,59,...,0.707956,21.251619,0.087284,10.317721,0.251522,-4.642145,110.561656,1.058531,0.707956,0.221992
112554,2024,115,1478,70,1447,56,H,0,26,59,...,0.710244,21.356865,0.088201,10.199289,0.249670,-3.545285,110.443756,1.059431,0.710244,0.219110
112716,2024,117,1478,74,1384,58,A,0,24,45,...,0.710083,21.670217,0.090078,10.330056,0.254415,-2.605008,110.112167,1.071831,0.710083,0.212538
112835,2024,121,1478,82,1192,61,H,0,27,61,...,0.710913,21.794753,0.090183,10.410675,0.255867,-1.465883,110.200255,1.075659,0.710913,0.213876


In [8]:
all_games_normalized['Win'] = (all_games_normalized['Score'] > all_games_normalized['OpponentScore']).astype(int)

# Assuming all_games_normalized has the 'Win' column updated based on scores

def rolling_wins(group, window_size):
    # Use the rolling function to sum 'Win' over the past 'window_size' games, filling NaNs with 0
    # The rolling window is not allowed to cross over into the next season's games
    wins = group['Win'].rolling(window=window_size, min_periods=1).sum().shift()
    return wins.fillna(0)

# Apply the rolling_wins function for each window size and create new columns in the DataFrame
window_sizes = range(1,35)
for window_size in window_sizes:
    column_name = f'last{window_size}'
    all_games_normalized[column_name] = all_games_normalized.groupby(['TeamID', 'Season']).apply(rolling_wins, window_size).reset_index(level=[0,1], drop=True)

# Ensure the DataFrame is still sorted by TeamID, Season, and DayNum
all_games_normalized.sort_values(by=['TeamID', 'Season', 'DayNum'], inplace=True)

# After this, all_games_normalized will include the new features 'last5', 'last10', and 'last15' for each game


/tmp/ipykernel_19/3144590169.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_games_normalized['Win'] = (all_games_normalized['Score'] > all_games_normalized['OpponentScore']).astype(int)
/tmp/ipykernel_19/3144590169.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_games_normalized[column_name] = all_games_normalized.groupby(['TeamID', 'Season']).apply(rolling_wins, window_size).reset_index(level=[0,1], drop=True)
/tmp/ipykernel_19/3144590169.py:15: Per

In [9]:
all_games_normalized

,Season,DayNum,TeamID,Score,OpponentID,OpponentScore,Loc,OT,FGM,FGA,...,last25,last26,last27,last28,last29,last30,last31,last32,last33,last34
169822,2014,5,1101,75,1182,94,A,0,22,63,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169892,2014,7,1101,47,1382,75,A,0,14,40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169982,2014,9,1101,44,1268,67,A,0,17,54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
170149,2014,13,1101,41,1234,103,A,0,14,54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
170249,2014,15,1101,64,1395,71,A,0,19,50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226820,2024,110,1478,58,1192,68,H,0,21,59,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
112554,2024,115,1478,70,1447,56,H,0,26,59,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
112716,2024,117,1478,74,1384,58,A,0,24,45,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
112835,2024,121,1478,82,1192,61,H,0,27,61,...,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0


In [10]:
massey.columns

Index(['Season', 'RankingDayNum', 'SystemName', 'TeamID', 'OrdinalRank'], dtype='object')

In [11]:
massey['SystemName'].unique()

array(['SEL', 'AP', 'BIH', 'DUN', 'ENT', 'GRN', 'IMS', 'MAS', 'MKV',
       'MOR', 'POM', 'RPI', 'SAG', 'SAU', 'SE', 'STR', 'USA', 'WLK',
       'WOB', 'BOB', 'DWH', 'ERD', 'ECK', 'BRZ', 'ARG', 'RTH', 'WOL',
       'HOL', 'COL', 'DOL', 'GC', 'GRS', 'HER', 'TSR', 'MIC', 'WTE', 'BD',
       'MGY', 'RAG', 'CNG', 'SIM', 'DES', 'JON', 'LYN', 'NOR', 'RM',
       'REI', 'ACU', 'BCM', 'CMV', 'SAP', 'DC', 'KLK', 'WIL', 'ROH',
       'RIS', 'REN', 'SCR', 'DOK', 'PIG', 'KPK', 'PKL', 'TRX', 'MB',
       'JCI', 'PH', 'LYD', 'KRA', 'RTR', 'UCS', 'ISR', 'CPR', 'BKM',
       'JEN', 'REW', 'STH', 'SPW', 'RSE', 'PGH', 'CPA', 'RTB', 'TOL',
       'HKB', 'BPI', 'TW', 'NOL', 'DC2', 'DCI', 'OMY', 'EBB', 'LMC', 'RT',
       'KEL', 'KMV', 'RTP', 'TMR', 'AUS', 'ROG', 'BP5', 'PTS', 'KOS',
       'PEQ', 'ADE', 'BNM', 'CJB', 'BUR', 'HAT', 'MSX', 'BBT', '7OT',
       'SFX', 'EBP', 'TBD', 'CRO', 'D1A', 'TPR', 'BLS', 'DII', 'KBM',
       'TRP', 'LOG', 'SP', 'STF', 'WMR', 'PPR', 'MPI', 'STS', 'UPS',
       'SPR', 'Mv

In [12]:
# Assuming 'massey' is your DataFrame with rankings
# and 'all_games_normalized' is your main games DataFrame

# We'll first ensure that both DataFrames are sorted properly
massey.sort_values(by=['Season', 'TeamID', 'RankingDayNum'], inplace=True)
all_games_normalized.sort_values(by=['Season', 'TeamID', 'DayNum'], inplace=True)

# Merge asof to get the most recent ranking for each game
# 'asof_merge' requires sorted DataFrames
rankings_for_games = pd.merge_asof(
    all_games_normalized.sort_values('DayNum'),
    massey.sort_values('RankingDayNum'),
    by=['Season', 'TeamID'],
    left_on='DayNum',
    right_on='RankingDayNum',
    direction='backward'
)

# Now, 'rankings_for_games' contains the most recent ranking for each game
# But we need to pivot this so each SystemName has its own column
# Pivot the table to have SystemNames as columns and fill missing rankings with an appropriate value or NaN
pivot_rankings = rankings_for_games.pivot_table(
    index=['Season', 'TeamID', 'DayNum'],  # Preserving the game identifier
    columns='SystemName',  # Creating a column for each ranking system
    values='OrdinalRank'
).reset_index()

# Optionally, fill NaN values if needed, e.g., with forward fill within each season and team
pivot_rankings.fillna(method='ffill', inplace=True)
# Merge the pivoted rankings back onto the original games DataFrame
all_games_with_rankings = pd.merge(
    all_games_normalized,
    pivot_rankings,
    on=['Season', 'TeamID', 'DayNum'],
    how='left'
)

all_games_with_rankings = pd.merge(
    all_games_with_rankings,
    pivot_rankings.rename(columns={"TeamID":"OpponentID"}),
    on=['Season', 'OpponentID', 'DayNum'],
    how='left'
)

# 'all_games_with_rankings' now includes a column for each ranking system with the most recent ranking for each game


/tmp/ipykernel_19/805657548.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pivot_rankings.fillna(method='ffill', inplace=True)


In [13]:
# Correctly use forward fill without 'method' parameter
all_games_with_rankings.sort_values(by=['Season', 'TeamID', 'DayNum'], inplace=True)
all_games_with_rankings.ffill(inplace=True)

# Fill remaining NaNs in ranking columns with the median value for each column, avoiding chained assignment
ranking_columns = [col for col in all_games_with_rankings.columns if '_x' in col or '_y' in col]

# Compute median values for ranking columns
median_ranks = all_games_with_rankings[ranking_columns].median()

# Use DataFrame.update() to fill NaNs with median values directly on the original DataFrame
all_games_with_rankings.update(all_games_with_rankings[ranking_columns].fillna(median_ranks))

In [14]:
all_games_with_rankings.columns

Index(['Season', 'DayNum', 'TeamID', 'Score', 'OpponentID', 'OpponentScore',
       'Loc', 'OT', 'FGM', 'FGA',
       ...
       'WIL_y', 'WLK_y', 'WLS_y', 'WMR_y', 'WMV_y', 'WOB_y', 'WOL_y', 'WTE_y',
       'YAG_y', 'ZAM_y'],
      dtype='object', length=581)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np
import pandas as pd

def preprocess_data(all_games_with_rankings, test_size=0.2, random_state=42):
    """
    Preprocesses the game data for neural network training.
    
    Parameters:
    - all_games_with_rankings: DataFrame containing game statistics and rankings.
    - test_size: Proportion of the dataset to include in the test split.
    - random_state: Controls the shuffling applied to the data before applying the split.
    
    Returns:
    - X_train_scaled: Scaled training features.
    - X_val_scaled: Scaled validation features.
    - y_train: Training target.
    - y_val: Validation target.
    - encoder: Fitted OneHotEncoder instance.
    - scaler: Fitted StandardScaler instance.
    """
    features = all_games_with_rankings.drop(['Win'], axis=1)  # Drop the target column
    target = all_games_with_rankings['Win'].values  # This is your target variable

    # One-hot encode the 'Type' column
    encoder = OneHotEncoder(sparse=False)
    type_encoded = encoder.fit_transform(features[['Type']])

    # Get one-hot encoded feature names
    type_feature_labels = encoder.get_feature_names_out(['Type'])

    # Concatenate the one-hot encoded 'Type' back with the rest of your numeric features
    numeric_features = features.drop(['Score','OpponentScore','Loc',
                                      'OT','FGM','FGA','FGM3','FGA3','FTM','FTA',
                                      'OR','DR','Ast','TO','Stl','Blk','PF','OpponentFGM',
                                      'OpponentFGA','OpponentFGM3','OpponentFGA3',
                                      'OpponentFTM','OpponentFTA','OpponentOR','OpponentDR',
                                      'OpponentAst','OpponentTO','OpponentStl','OpponentBlk',
                                      'OpponentPF','Type', 'FGPct', 'FG3Pct', 'FTPct',
       'OpponentFGPct', 'OpponentFG3Pct', 'OpponentFTPct', 'Possessions',
       'OpponentPossessions', 'OE', 'DE', 'TSP', 'eFG%', 'TOR', 'ORR',
       'DRR', 'AR', 'BlockRate', 'StealRate', 'FTR', 'NetRating', 'USG%', 'PPS', 'DRB%', 'ORB%'], axis=1).select_dtypes(include=[np.number])
    
    numeric_features2 = numeric_features.copy()
    merged = pd.merge(numeric_features, numeric_features2, left_on=['Season', 'DayNum', 'TeamID'], right_on=['Season', 'DayNum', 'OpponentID']).drop(columns= ["Season", "DayNum", 'TeamID_x', 'OpponentID_x', 'TeamID_y', 'OpponentID_y'])
    print(len(list(merged.columns)))

    numeric_features_array = merged.values
    features_encoded = np.concatenate([numeric_features_array, type_encoded], axis=1)
    

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(features_encoded, target, test_size=test_size, random_state=random_state)

    # Scale the numeric features (excluding the one-hot encoded 'Type' columns)
    scaler = StandardScaler()
    N = features_encoded.shape[1]  # Number of numeric features
    X_train_scaled = np.copy(X_train)
    X_val_scaled = np.copy(X_val)
    X_train_scaled[:, :N] = scaler.fit_transform(X_train[:, :N])
    X_val_scaled[:, :N] = scaler.transform(X_val[:, :N])

    return X_train_scaled, X_val_scaled, y_train, y_val, encoder, scaler


In [16]:
X_train_scaled, X_val_scaled, y_train, y_val, encoder, scaler = preprocess_data(all_games_with_rankings)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


1042


In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=256, step=32),
                    activation='relu', input_shape=(X_train_scaled.shape[1],)))
    
    for i in range(hp.Int('n_layers', 1, 5)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
                        activation='relu'))
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, default=0.5, step=0.1)))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


2024-03-19 03:08:38.886627: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 03:08:38.886766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 03:08:39.048528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_19/2323700927.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [18]:
import kerastuner as kt

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,  # Set more trials for a more thorough search.
    executions_per_trial=1,
    directory='my_dir',
    project_name='hparam_tuning3'
)

# Early stopping callback to prevent overfitting
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Start hyperparameter search
tuner.search(X_train_scaled, y_train,
             epochs=1,
             validation_data=(X_val_scaled, y_val),
             callbacks=[stop_early],
             verbose=1)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters:\n{best_hps.values}")

# Build the model with the best hyperparameters and train it on the data
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=1,  # You may adjust this based on the early stopping callback
                         verbose=1)

# Optionally, evaluate the best model
val_loss, val_accuracy = best_model.evaluate(X_val_scaled, y_val, verbose=0)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')


Trial 1 Complete [00h 00m 19s]
val_accuracy: 0.7529188394546509

Best val_accuracy So Far: 0.7529188394546509
Total elapsed time: 00h 00m 19s
Best hyperparameters:
{'units_input': 32, 'n_layers': 1, 'units_0': 96, 'dropout_0': 0.30000000000000004, 'learning_rate': 0.007120962612767163}
5728/5728 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.7347 - loss: 0.5259 - val_accuracy: 0.7218 - val_loss: 0.5353
Validation Loss: 0.5353077054023743, Validation Accuracy: 0.7217772603034973


In [19]:
teams = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MTeams.csv')

In [20]:
numeric_feature_names = list(all_games_with_rankings.drop(columns = ['Score','OpponentScore','Loc',
                                      'OT','FGM','FGA','FGM3','FGA3','FTM','FTA',
                                      'OR','DR','Ast','TO','Stl','Blk','PF','OpponentFGM',
                                      'OpponentFGA','OpponentFGM3','OpponentFGA3',
                                      'OpponentFTM','OpponentFTA','OpponentOR','OpponentDR',
                                      'OpponentAst','OpponentTO','OpponentStl','OpponentBlk',
                                      'OpponentPF','Type', 'Win', 'FGPct', 'FG3Pct', 'FTPct',
       'OpponentFGPct', 'OpponentFG3Pct', 'OpponentFTPct', 'Possessions',
       'OpponentPossessions', 'OE', 'DE', 'TSP', 'eFG%', 'TOR', 'ORR',
       'DRR', 'AR', 'BlockRate', 'StealRate', 'FTR', 'NetRating', 'USG%', 'PPS', 'DRB%', 'ORB%']).columns)

In [21]:
def matchup_predictor(team_a_id, team_b_id, season, day_num, model, encoder, scaler):
    combined_features = generate_game_features(team_a_id, team_b_id, season, day_num, encoder, scaler, numeric_feature_names)
    # Ensure combined_features is correctly shaped for model input
    combined_features_reshaped = scaler.transform(combined_features)
    #print(combined_features)
    win_probability = model.predict(combined_features_reshaped)[0]
    return win_probability

In [22]:
def generate_game_features(team_id_a, team_id_b, season, day_num, encoder, scaler, numeric_feature_names):
    
    # Existing code to filter all_games_with_rankings for a specific season, team, and before a certain day
    team_a_data = all_games_with_rankings[
        (all_games_with_rankings['Season'] == season) &
        (all_games_with_rankings['TeamID'] == team_a_id) &
        (all_games_with_rankings['DayNum'] < day_num)
    ]

    # New code to filter team_a_data to only include the row with the max DayNum
    if not team_a_data.empty:
        max_day_num_index = team_a_data['DayNum'].idxmax()
        team_a_data = team_a_data.loc[[max_day_num_index]]

    team_b_data = all_games_with_rankings[
        (all_games_with_rankings['Season'] == season) &
        ((all_games_with_rankings['TeamID'] == team_b_id)) &
        (all_games_with_rankings['DayNum'] < day_num)
    ]
    
    if not team_b_data.empty:
        max_day_num_index = team_b_data['DayNum'].idxmax()
        team_b_data = team_b_data.loc[[max_day_num_index]]

    if team_a_data.empty:
        raise ValueError(f"No data found for team ID {team_a_id} in season {season}.")
    if team_b_data.empty:
        raise ValueError(f"No data found for team ID {team_b_id} in season {season}.")

    # Assuming 'Type' is a column in team_data, create a DataFrame for encoding
    # Ensure this matches how 'Type' was handled during model training
    
    merged = pd.merge(team_a_data[numeric_feature_names], team_b_data[numeric_feature_names], on = "Season").drop(columns= ['Season', 'DayNum_x', 'TeamID_x', 'OpponentID_x', 'DayNum_y', 'TeamID_y', 'OpponentID_y'])

    type_df = team_a_data[['Type']].drop_duplicates()  # Adjust as necessary

    type_encoded = encoder.transform(type_df)

    # Calculate mean of numeric features
    numeric_means = merged[merged.columns].mean().to_frame().T
    #print(team_a_data[numeric_feature_names])

    # Concatenate encoded 'Type' with numeric features BEFORE scaling
    combined_features = np.concatenate([numeric_means, type_encoded], axis=1)

    # Scale the combined features
    #features_scaled = scaler.transform(combined_features)
    
    #print(team_id_a, team_id_b)
    return combined_features

In [23]:

season = 2024
day_num = 999
team_a_id = 1104
team_b_id = 1120

# Predicting the win probability for Team A against Team B
win_probability = matchup_predictor(team_a_id, team_b_id, season, day_num, best_model, encoder, scaler)

print(f"The probability of Team A (ID: {team_a_id}) winning against Team B (ID: {team_b_id}) is {win_probability[0]:.2}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
The probability of Team A (ID: 1104) winning against Team B (ID: 1120) is 0.44


In [24]:
def predict_game_winner(team_a_id, team_b_id, season, daynum, model, encoder, scaler):
    """
    Wrapper for the matchup_predictor to return the winning team based on the predicted probability.
    Incorporates the day number into the prediction.
    """
    # Assuming the matchup_predictor function returns a probability of team_a winning
    print(team_a_id, team_b_id)
    win_probability = matchup_predictor(team_a_id, team_b_id, season, daynum, model, encoder, scaler)
    # Randomly determine the game outcome based on win probability
    return team_a_id if np.random.rand() < win_probability else team_b_id

In [25]:
structure = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeedRoundSlots.csv').drop(columns = ["EarlyDayNum", "LateDayNum"])
seeds_csv = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv')
seeds_csv = seeds_csv[seeds_csv["Tournament"] == 'M']

In [26]:
round_slots = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySlots.csv')
round_slots = round_slots[round_slots["Season"] == 2024].drop(columns = ["Season"]).reset_index().drop(columns = ["index"])

In [27]:
structure[structure["GameSlot"] == "R1W1"]

,Seed,GameRound,GameSlot
0,W01,1,R1W1
174,W16,1,R1W1
181,W16a,1,R1W1
188,W16b,1,R1W1


In [28]:
import pandas as pd

def run_tournament_simulation(seeds, round_slots, model, encoder, scaler, season, daynum, tournament_type, bracket_num):
    seeds_dict = seeds.set_index('Seed')['TeamID'].to_dict()
    # Initialize a dictionary to map slots to team IDs dynamically
    slot_to_teamID = {seed: teamID for seed, teamID in seeds_dict.items()}
    
    winners = []
    for _, row in round_slots.iterrows():
        # Use the updated dynamic mapping for team IDs
        strong_team_id = slot_to_teamID.get(row['StrongSeed'])
        weak_team_id = slot_to_teamID.get(row['WeakSeed'])
        
        if not strong_team_id or not weak_team_id:
            print("E: " + row['StrongSeed'] + " " + row['WeakSeed'])
            continue
        
        winner_id = predict_game_winner(strong_team_id, weak_team_id, season, daynum, model, encoder, scaler)
        
        # Determine the winner's seed and update the dynamic mapping for the next round
        winner_seed = next((seed for seed, id in seeds_dict.items() if id == winner_id), None)
        slot_to_teamID[row['Slot']] = winner_id  # Update for next round's matchups
        
        winners.append((row['Slot'], winner_seed))
    
    # Convert the winners list to a DataFrame for submission
    winners_df = pd.DataFrame(winners, columns=['Slot', 'Team'])
    winners_df['Bracket'] = bracket_num
    winners_df['Tournament'] = tournament_type
    winners_df.reset_index(inplace=True, drop=True)
    winners_df.index.name = 'RowId'

    return winners_df

def simulate_n_brackets(n, seeds, round_slots, model, encoder, scaler, season, daynum, tournament_type='M'):
    all_brackets = pd.DataFrame()
    
    for bracket_num in range(1, n + 1):
        bracket_results = run_tournament_simulation(seeds, round_slots, model, encoder, scaler, season, daynum, tournament_type, bracket_num)
        bracket_results['Bracket'] = bracket_num  # Tag each row with its bracket number
        
        all_brackets = pd.concat([all_brackets, bracket_results], ignore_index=True)
    
    all_brackets.reset_index(inplace=True)
    all_brackets.rename(columns={'index': 'RowId'}, inplace=True)
    
    return all_brackets


In [29]:
round_slots

,Slot,StrongSeed,WeakSeed
0,R1W1,W01,W16
1,R1W2,W02,W15
2,R1W3,W03,W14
3,R1W4,W04,W13
4,R1W5,W05,W12
...,...,...,...
62,R6CH,R5WX,R5YZ
63,X16,X16a,X16b
64,Y10,Y10a,Y10b
65,Y16,Y16a,Y16b


In [30]:
results = simulate_n_brackets(20, seeds_csv, round_slots, best_model, encoder, scaler, 2024, 999, 'M')

1163 1391
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1235 1355
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1228 1287
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1120 1463
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1361 1412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1140 1182
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1450 1179
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1194 1321
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1314 1224
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1112 1253
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1124 1159
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1104 1158
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1388 1213
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1155 1307
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1173 1305
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1280 1277
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1345 1212
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1397 1389
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1166 1103
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1242 1359
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1211 1270
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20

In [31]:
teams[teams["TeamID"] == 1222]

,TeamID,TeamName,FirstD1Season,LastD1Season
121,1222,Houston,1985,2024


In [32]:
results[results["Slot"].str.contains("R4")][:12]

,RowId,Slot,Team,Bracket,Tournament
56,56,R4W1,W07,1,M
57,57,R4X1,X13,1,M
58,58,R4Y1,Y11,1,M
59,59,R4Z1,Z01,1,M
119,119,R4W1,W05,2,M
120,120,R4X1,X10,2,M
121,121,R4Y1,Y11,2,M
122,122,R4Z1,Z03,2,M
182,182,R4W1,W07,3,M
183,183,R4X1,X02,3,M


In [33]:
round_slots[:30]

,Slot,StrongSeed,WeakSeed
0,R1W1,W01,W16
1,R1W2,W02,W15
2,R1W3,W03,W14
3,R1W4,W04,W13
4,R1W5,W05,W12
5,R1W6,W06,W11
6,R1W7,W07,W10
7,R1W8,W08,W09
8,R1X1,X01,X16
9,R1X2,X02,X15
